In [1]:
import os
from pathlib import Path
from pymatgen.io.cif import CifWriter
from mp_api.client import MPRester
import pandas as pd
import json
import ormsgpack as mp
import numpy as np

ids = pd.read_csv("./data/material-data/mp-ids-46744.csv")
ids = list(pd.Series(ids[ids.columns[0]]))
print(len(ids))


# def save_structure_to_cif(structure, output_file):
#     with open(output_file, "w") as f:
#         cif_writer = CifWriter(structure)
#         cif_writer.write_file(f.name)


# def save_properties_to_json(properties: dict, output_file):
#     with open(output_file, "w") as f:
#         json.dump(properties, f)


cifs_file = "./data/root/data/cifs.bin"


def save_structures_to_bin(structures: dict, output_file=cifs_file):
    with open(output_file, "wb") as f:
        f.write(mp.packb(structures, option=mp.OPT_NON_STR_KEYS))


props_file = "./data/root/data/props.bin"


def save_properties_to_bin(properties: dict, output_file=props_file):
    with open(output_file, "wb") as f:
        f.write(mp.packb(properties, option=mp.OPT_NON_STR_KEYS))


# fields=['formation_energy_per_atom']
fields = [
    "energy_per_atom",
    "formation_energy_per_atom",
    "band_gap",
    "efermi",
    "k_voigt",
    "k_reuss",
    "k_vrh",
    "g_voigt",
    "g_reuss",
    "g_vrh",
    "homogeneous_poisson",
]

# mids=[]
# dmids=[]
# mids1 = []
# print(len(set(ids)))
tdocs = []
def download_all():
    # global ids,mids,dmids,mids1,tdocs
    with MPRester("os.environ.get('MP_API_KEY')") as mpr:
        pass
        # docs = mpr.materials.search(task_ids=ids, fields=["material_id"])
        # mids = [str(d.material_id) for d in docs]
        # print(len(mids),len(ids))

        # ddocs = mpr.materials.search(task_ids=ids, fields=["material_id"],deprecated=True)
        # dmids = [str(d.material_id) for d in ddocs]
        # print(len(dmids),len(mids)+len(dmids),len(ids))
        # mids1=set(mids).union(set(dmids))
        # tdocs = mpr.tasks.search(list(set(ids).difference(mids1)))
        # tmids = [str(d) for d in tdocs]
        # print(len(tmids),len(ids))
        
        # s = set(ids)
        # normal_ids = mpr.materials.search(
        #     task_ids=ids, fields=["task_ids", "material_id"]
        # )
        # normal_ids += mpr.materials.search(
        #     deprecated=True, task_ids=ids, fields=["task_ids", "material_id"]
        # )
        # tsk_ids = []
        # for x in normal_ids:
        #     tsk_ids += [str(y) for y in x.task_ids if y in s]
        # # deprecated_ids=mpr.materials.search(deprecated=True, task_ids=ids, fields=['task_ids','material_id'])
        # # deprecated_idss = [str([y for y in x.task_ids if y in s][0]) for x in deprecated_ids]
        # total_missing_ids = set(ids).difference(set(tsk_ids))
        # # print(normal_ids[0].model_dump().keys())
        # # docs = mpr.materials.tasks.search(ids[:10])
        # # print(len(docs[0].calcs_reversed))
        # # print(docs[0].calcs_reversed[0].model_dump().task_name)
        # print(len(total_missing_ids), total_missing_ids)
        # missing_ids = mpr.summary.search(
        #     material_ids=total_missing_ids
        # ) + mpr.summary.search(deprecated=True, material_ids=total_missing_ids)
        # missing_tsk_ids = []
        # for x in normal_ids:
        #     missing_tsk_ids += [str(y) for y in x.task_ids if y in s]
        # missing_entries = mpr.materials.summary.search(material_ids=total_missing_ids)
        # missing_entries += mpr.materials.summary.search(
        #     deprecated=True, material_ids=total_missing_ids
        # )
        # missing_entriess = [
        #     str([y for y in x.task_ids if y in s][0]) for x in missing_entries
        # ]

        # print(
        #     len(set(total_missing_ids).difference(set(missing_entriess))),
        #     set(total_missing_ids).difference(set(missing_entriess)),
        # )
        # docs = mpr.materials.summary.search(
        #     material_ids=ids, fields=fields + ["structure", "material_id"]
        # )
        # # deprecated_docs = mpr.materials.summary.search(deprecated=True, material_ids=deprecated_ids, fields=fields+['structure','material_id'])
        # # print('docs:',len(docs))
    # props = dict()
    # cifs = dict()
    # for d in docs:
        # output_file = f"./data/root/{d.material_id}.cif"
        # save_structure_to_cif(d.structure, output_file)
        # save_properties_to_json(d.model_dump(include=fields),f"props/{d.material_id}.json")
        # cifs[d.material_id] = d.structure.as_dict()
        # props[d.material_id] = d.model_dump(include=fields)
    # save_structures_to_bin(cifs)
    # save_properties_to_bin(props)
    # uncorrected_energy_per_atom energy_per_atom formation_energy_per_atom band_gap efermi
    #  see https://docs.materialsproject.org/methodology/materials-methodology/elasticity/ k <-> bulk, g <-> shear
    # k_vrh g_vrh homogeneous_poisson


# download_all()


/home/nodoteve/apps/pythom/miniconda/envs/cgcnn2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


46743


In [16]:
with MPRester("os.environ.get('MP_API_KEY')") as mpr:
    docs = mpr.materials.search(task_ids=ids, fields=['material_id', 'task_id'])
    mids = [str(d.material_id) for d in docs]
    ddocs = mpr.materials.search(task_ids=ids, fields=['material_id', 'task_id'],deprecated=True)
    dmids = [str(d.material_id) for d in ddocs]

    # # get initial tids that are not covered
    smids = list(set(mids).union(set(dmids)))

    sdocs=mpr.materials.search(material_ids=smids,fields=['material_id','task_ids'])
    smids = [(str(d.material_id),d.task_ids) for d in sdocs]
    dsdocs=mpr.materials.search(material_ids=smids,fields=['material_id','task_ids'],deprecated=True)
    dsdmids = [(str(d.material_id),d.task_ids) for d in dsdocs]
    sids=set(ids)
    sd=dict(smids)
    sd.update(dsdmids)
    p = [(k,[t for t in v if t in sids]) for k,v in sd.items()]
    count= 0
    for q in p:
        if len(q[1])>1:
            count+=len(q[1])-1
    print('Duplicates in almost complete dataset(?):', count)
    w = set()
    for x in p:
        w=w.union(set(x[1]))
    # flatten
    diff=sids.difference(w)
    print(len(diff))
    print(diff)

    diffdocs=mpr.materials.search(material_ids=list(diff),fields=['material_id','task_ids'])
    diffdocs=mpr.materials.search(task_ids=list(diff),fields=['material_id','task_ids'])
    print(diffdocs[0])
    # ->IndexError: list index out of range
    
    diffdocs=mpr.materials.search(material_ids=list(diff),fields=['material_id','task_ids'],deprecated=True)
    print(diffdocs[4])
    diffdocs=mpr.materials.search(task_ids=list(diff),fields=['material_id','task_ids'],deprecated=True)
    print(diffdocs[4])
    mps = [(str(d.material_id),d.task_ids) for d in diffdocs]
    for x in mps:
        w=w.union(set(x[1]))
    ndiff=sids.difference(w)
    print(len(ndiff))
    
    mpids = [x[0] for x in mps] + [x[0] for x in p]
    deduplmpids=list(set(mpids))
    print(len(deduplmpids),len(mpids))

    # now get structures and props:
    strprdocs=mpr.materials.search(material_ids=deduplmpids,fields=fields + ["structure", "material_id"])
    dstrprdocs=mpr.materials.search(material_ids=deduplmpids,fields=fields + ["structure", "material_id"],deprecated=True)
    docs = strprdocs+dstrprdocs
    checkdupl=[x.material_id for x in docs]
    print(len(set(checkdupl)),len(checkdupl))
    props = dict()
    cifs = dict()
    # for d in docs:
        # output_file = f"./data/root/{d.material_id}.cif"
        # save_structure_to_cif(d.structure, output_file)
        # save_properties_to_json(d.model_dump(include=fields),f"props/{d.material_id}.json")
        # cifs[d.material_id] = d.structure.as_dict()
        # props[d.material_id] = d.model_dump(include=fields)
    # save_structures_to_bin(cifs)
    # save_properties_to_bin(props)

/home/nodoteve/apps/pythom/miniconda/envs/cgcnn2/lib/python3.11/site-packages/mp_api/client/mprester.py:227: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


set()
50012 50418


Retrieving MaterialsDoc documents: 100%|██████████| 3717/3717 [00:20<00:00, 180.86it/s] 

50012 50012


In [ ]:
# compress with zstd
import zstandard as zstd


def compress_all():
    with open(cifs_file, "rb") as f:
        res = zstd.compress(f.read(), level=10)
    with open("./data/root/cifs.zstd", "wb") as f:
        f.write(res)
    with open(props_file, "rb") as f:
        res = zstd.compress(f.read(), level=10)
    with open("./data/root/props.zstd", "wb") as f:
        f.write(res)


# compress_all()


def uncompress_all():
    with open("./data/root/cifs.zstd", "rb") as f:
        res = zstd.decompress(f.read())
    with open(cifs_file, "wb") as f:
        f.write(res)
    with open("./data/root/props.zstd", "rb") as f:
        res = zstd.decompress(f.read())
    with open(props_file, "wb") as f:
        f.write(res)


# uncompress_all()

In [ ]:
# print data statistics (Table 1)
# basedir = './data/root/props/'
basedir = './data/root/data/'
# propfiles = os.listdir(basedir) 
def load_properties_from_json(file):
    filename = os.path.basename(file)
    with open(basedir+file, 'rb') as f:
        properties=json.load(f)
        return (filename, properties)
    return None
def load_properties_from_bin(file):
    filename = os.path.basename(file)
    with open(file, 'rb') as f:
        properties=mp.unpackb(f.read())
        return properties
    return None
# propdata = dict()
# for file in propfiles:
#     name,prop=load_properties_from_json(file) 
#     for key in prop.keys():
#         propdata.setdefault(key,[])
#         propdata[key].append(prop[key])

f=pd.DataFrame(load_properties_from_bin(basedir+'props.bin')).transpose()
import numpy as np
for moduli in ['k_voigt','k_reuss',	'k_vrh', 'g_voigt',	'g_reuss', 'g_vrh']:
    f[moduli]=np.log(f[moduli])
f.describe()

/home/nodoteve/apps/pythom/miniconda/envs/cgcnn2/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,energy_per_atom,formation_energy_per_atom,band_gap,efermi,k_voigt,k_reuss,k_vrh,g_voigt,g_reuss,g_vrh,homogeneous_poisson
count,46295.000000,46295.000000,46295.000000,46295.000000,5535.000000,5520.000000,5533.000000,5494.000000,5336.000000,5444.000000,5548.000000
mean,-6.422922,-1.709291,1.398688,2.900727,4.437200,4.347801,4.402111,3.727987,3.525721,3.632691,0.294389
std,1.769580,1.056798,1.660049,2.679149,0.828071,0.961747,0.864749,0.868668,1.073388,0.930323,0.363060
min,-14.076122,-4.499302,0.000000,-14.017281,-0.058679,-1.673020,-0.117437,-1.944802,-6.026329,-2.208196,-13.047463
25%,-7.674900,-2.510649,0.000000,1.079865,3.955201,3.873712,3.905877,3.242710,2.983668,3.107202,0.250640
50%,-6.675442,-1.863545,0.717000,2.638274,4.628538,4.592086,4.613223,3.861791,3.704819,3.771601,0.291793
75%,-5.258740,-0.736119,2.493250,4.614450,5.064225,5.048719,5.052382,4.346580,4.277646,4.309973,0.331428
max,-0.005272,4.309211,16.586400,17.758767,6.687777,6.639926,6.664137,6.055889,7.600026,6.912284,19.038342


In [ ]:
# prepare for train
def set_property_to_ids(property):
    f[property].to_csv('./data/root/data/id_prop.csv',index=True,header=False)
set_property_to_ids('energy_per_atom')

In [ ]:
# research len diff
initial_ids= pd.read_csv('./data/material-data/mp-ids-46744.csv')
initial_ids=list(pd.Series(initial_ids[initial_ids.columns[0]]))
diff=set(ids).difference(set(f.index))
print(len(diff),diff)


10324 {'mp-566537', 'mp-541672', 'mp-565905', 'mp-773396', 'mp-779996', 'mp-762566', 'mp-763154', 'mp-695787', 'mp-510766', 'mp-561884', 'mp-775562', 'mp-767292', 'mp-777554', 'mp-566277', 'mp-764168', 'mp-766155', 'mp-774025', 'mp-868122', 'mp-765447', 'mp-770159', 'mp-761358', 'mp-504312', 'mp-867570', 'mp-868385', 'mp-780038', 'mp-764533', 'mp-865135', 'mp-771542', 'mp-25788', 'mp-763681', 'mp-769814', 'mp-765684', 'mp-566981', 'mp-780152', 'mp-765657', 'mp-566796', 'mp-771603', 'mp-764674', 'mp-566127', 'mp-771497', 'mp-625416', 'mp-763679', 'mp-868756', 'mp-762431', 'mp-762039', 'mp-24922', 'mp-773338', 'mp-761412', 'mp-775035', 'mp-554962', 'mp-565656', 'mp-771278', 'mp-24970', 'mp-767332', 'mp-25083', 'mp-763823', 'mp-771745', 'mp-779936', 'mp-771684', 'mp-764576', 'mp-772450', 'mp-765545', 'mp-764024', 'mp-761605', 'mp-766672', 'mp-761326', 'mp-771101', 'mp-775033', 'mp-779214', 'mp-767731', 'mp-641500', 'mp-565897', 'mp-778296', 'mp-706279', 'mp-763833', 'mp-766850', 'mp-77108

In [ ]:
# train on all properties


In [7]:
import numpy as np
from subprocess import run
import shutil as st
resdict=dict()
maedict=dict()
reference_csv = {'mp-ids-3402.csv':['k_voigt',
'k_reuss',
'k_vrh',
'g_voigt',
'g_reuss', 'g_vrh',
'homogeneous_poisson'],
'mp-ids-27430.csv':['band_gap']
,'mp-ids-46744.csv':['energy_per_atom',
'formation_energy_per_atom',
'efermi']
}

def set_property_to_ids(df,property):
    df[property].to_csv('./data/root/data/id_prop.csv',index=True,header=False)
def prepare_csv(csv, df,prop):
    st.move('./data/material-data/'+csv,'./data/root/data/id_prop.csv')
    set_property_to_ids(df,prop)
def download_csv(csv):
    ids = pd.read_csv('./data/material-data/'+csv)
    ids = list(pd.Series(ids[ids.columns[0]]))
    s = set(ids)
    with MPRester("os.environ.get('MP_API_KEY')") as mpr:
        normal_ids = mpr.materials.search(task_ids=ids, fields=['task_ids','material_id'])
        depr_normal_ids = mpr.materials.search(deprecated=True, task_ids=ids, fields=['task_ids','material_id'])
        tsk_ids = []
        for x in normal_ids+depr_normal_ids:
            tsk_ids+=[(str(y),x.material_id) for y in x.task_ids if y in s]
        tsk_ids=np.array(tsk_ids).T
        total_missing_ids = set(ids).difference(set(tsk_ids[1]))
        # tsk_ids=dict(tsk_ids)
        print(len(tsk_ids[1]), len(tsk_ids[1])-len(set(tsk_ids[1])))
        docs = mpr.materials.summary.search(material_ids=list(tsk_ids[1]), fields=fields+['structure','material_id'])
        docs += mpr.materials.summary.search(deprecated=True,material_ids=list(tsk_ids[1]), fields=fields+['structure','material_id'])

        print(len(total_missing_ids), total_missing_ids)
        missing_ids=mpr.summary.search(material_ids=total_missing_ids)
        depr_missing_ids=mpr.summary.search(deprecated=True,material_ids=total_missing_ids)
        missing_tsk_ids = []
        for x in missing_ids+depr_missing_ids:
            missing_tsk_ids+=[(str(y),x.material_id) for y in x.task_ids if y in s]
        missing_tsk_ids=np.array(missing_tsk_ids).T
        # missing_tsk_ids=dict(missing_tsk_ids)
        missing_docs = mpr.materials.summary.search(material_ids=list(missing_tsk_ids[1]), fields=fields+['structure','material_id'])
        missing_docs += mpr.materials.summary.search(deprecated=True,material_ids=list(missing_tsk_ids[1]), fields=fields+['structure','material_id'])
    props=dict()
    cifs=dict()
    for d in docs:
        cifs[tsk_ids[d.material_id]]=d.structure.as_dict()
        props[d.material_id]=d.model_dump(include=fields)
    save_structures_to_bin(cifs)
    save_properties_to_bin(props)
download_csv('mp-ids-46744.csv')
for csv in reference_csv.keys():
    df = download_csv(csv)

    for prop in reference_csv['csv']:  
        prepare_csv(csv,df,prop)
        
        res = run('conda run -n cgcnn2 python main.py --train-ratio 0.6 --val-ratio 0.2 --test-ratio 0.2 ./data/root/data/',
                   capture_output=True,text=True, shell=True)
        # result = run(['ls'], capture_output=True, text=True)
        # Write the contents of the stdout (standard output) to the file.
        resdict[prop]=res.stdout
        maedict[prop]=resdict[prop].split('** MAE ')[-1]
        print(prop,maedict[prop])
    
        st.move('checkpoint.pth.tar','./trained/'+prop+'_check.pth.tar')
        st.move('model_best.pth.tar','./trained/'+prop+'_best.pth.tar')
        st.move('test_results.csv','./trained/'+prop+'_results.csv')

# for prop in fields:
#     set_property_to_ids(prop)
#     res = run('conda run -n cgcnn2 python main.py --train-ratio 0.6 --val-ratio 0.2 --test-ratio 0.2 ./data/root/data/',
#                capture_output=True,text=True, shell=True)
#     # result = run(['ls'], capture_output=True, text=True)
#     # Write the contents of the stdout (standard output) to the file.
#     resdict[prop]=res.stdout
#     maedict[prop]=resdict[prop].split('** MAE ')[-1]
#     print(prop,maedict[prop])

#     st.move('checkpoint.pth.tar','./trained/'+prop+'_check.pth.tar')
#     st.move('model_best.pth.tar','./trained/'+prop+'_best.pth.tar')
#     st.move('test_results.csv','./trained/'+prop+'_results.csv')
with open('train_outputs.json','w') as f:
    json.dump(resdict,f)
with open('train_maes.json','w') as f:
    json.dump(maedict,f)


NameError: name 'pd' is not defined